In [2]:
import sqlite3
import pandas as pd
import seaborn as sns
import json
import numpy as np
import os
from sams.utils import load_data
from sams.config import datasets

2025-08-07 12:15:55.730 | INFO     | sams.config:<module>:15 - PROJ_ROOT path is: C:\Users\Admin\Documents\GitHub\sams
2025-08-07 12:15:55.794 | WARNING  | sams.config:<module>:83 - Google MAPS API key not found, using Nominatim geocoder
2025-08-07 12:15:55.796 | INFO     | sams.config:<module>:92 - Loaded 0 geocodes from cache


In [3]:
from pathlib import Path
DATA_DIR = Path("C:/Users/Admin/Documents/GitHub/sams/data")
RAW_DATA_DIR = DATA_DIR / "raw"
hss_df = pd.read_parquet(RAW_DATA_DIR / "hss_data.pq", engine='pyarrow')

In [ ]:
option

Index(['aadhar_no', 'barcode', 'academic_year', 'gender', 'nationality',
       'state', 'district', 'block', 'pin_code', 'social_category',
       'examination_boardofthe_highest_qualification',
       'board_exam_namefor_highest_qualification', 'examination_type',
       'year_of_passing', 'total_marks', 'secured_marks', 'percentage',
       'compartmental_status', 'hss_option_details'],
      dtype='object')

In [4]:
hss_df.shape

(3453401, 19)

In [5]:
hss_df.columns

Index(['aadhar_no', 'barcode', 'academic_year', 'gender', 'nationality',
       'state', 'district', 'block', 'pin_code', 'social_category',
       'examination_boardofthe_highest_qualification',
       'board_exam_namefor_highest_qualification', 'examination_type',
       'year_of_passing', 'total_marks', 'secured_marks', 'percentage',
       'compartmental_status', 'hss_option_details'],
      dtype='object')

In [8]:
import pandas as pd
import numpy as np

def check_missing(df: pd.DataFrame) -> pd.DataFrame:
    """Return a summary of missing/blank/null-like values in each column."""
    summary = pd.DataFrame({
        "dtype": df.dtypes,
        "n_rows": len(df),
        "n_missing": df.isna().sum(),
        "n_blank": df.apply(lambda col: col.astype(str).str.fullmatch(r"\s*").sum()),
        "n_null_like": df.apply(lambda col: col.isin(["NA", "na", "N/A", "n/a", "NULL", "null"]).sum()),
    })
    summary["missing_pct"] = (summary["n_missing"] / summary["n_rows"]) * 100
    summary["blank_pct"] = (summary["n_blank"] / summary["n_rows"]) * 100
    summary["null_like_pct"] = (summary["n_null_like"] / summary["n_rows"]) * 100
    return summary.sort_values(by=["n_missing", "n_blank", "n_null_like"], ascending=False)

missing_summary = check_missing(hss_df)
missing_summary


,dtype,n_rows,n_missing,n_blank,n_null_like,missing_pct,blank_pct,null_like_pct
pin_code,Int32,3453401,183836,0,0,5.323332,0.000000,0.0
aadhar_no,string[python],3453401,45000,385857,0,1.303063,11.173246,0.0
barcode,string[python],3453401,0,0,0,0.000000,0.000000,0.0
academic_year,Int16,3453401,0,0,0,0.000000,0.000000,0.0
gender,category,3453401,0,0,0,0.000000,0.000000,0.0
nationality,category,3453401,0,0,0,0.000000,0.000000,0.0
state,category,3453401,0,0,0,0.000000,0.000000,0.0
district,category,3453401,0,0,0,0.000000,0.000000,0.0
block,category,3453401,0,0,0,0.000000,0.000000,0.0
social_category,category,3453401,0,0,0,0.000000,0.000000,0.0


In [5]:
db_path = datasets["sams"]["path"]
conn = sqlite3.connect(db_path)

query = """
SELECT
    academic_year,
    aadhar_no,
    barcode,
    module,
    option_data
FROM students
"""

df = pd.read_sql_query(query, conn)
conn.close()

df.head()

,academic_year,aadhar_no,barcode,module,option_data
0,2020,NA,20S00045,PDIS,[]
1,2020,NA,20S00031,PDIS,[]
2,2020,NA,20S000532,PDIS,"[{""Phase"": ""1"", ""Option_No"": ""1"", ""Institute_N..."
3,2020,NA,20S0008095,PDIS,[]
4,2020,NA,20S000528,PDIS,[]


In [ ]:
df

In [6]:
import pandas as pd

# Separate data by module
df_iti = df[df['module'] == 'ITI']
df_diploma = df[df['module'] == 'Diploma']
df_pdis = df[df['module'] == 'PDIS']

# ================= OVERVIEW SUMMARY =================

total_iti_students = df_iti['aadhar_no'].nunique()
total_diploma_students = df_diploma['aadhar_no'].nunique()
total_pdis_students = df_pdis['barcode'].nunique()
total_hss_students  = hss_df['aadhar_no'].nunique()

print("========== [STUDENT OVERVIEW] ==========")
print(f"Total ITI Students     : {total_iti_students:,}")
print(f"Total Diploma Students : {total_diploma_students:,}")
print(f"Total PDIS Students    : {total_pdis_students:,}")
print(f"Total HSS Students     : {total_hss_students:,}")
print()

# ================= YEAR-WISE SUMMARY =================

# Count unique students by year for each module
iti_counts = df_iti.groupby('academic_year')['aadhar_no'].nunique().rename('ITI')
diploma_counts = df_diploma.groupby('academic_year')['aadhar_no'].nunique().rename('Diploma')
pdis_counts = df_pdis.groupby('academic_year')['barcode'].nunique().rename('PDIS')
hss_counts = hss_df.groupby('academic_year')['aadhar_no'].nunique().rename('HSS')


# Combine all module counts
combined = pd.concat([iti_counts, diploma_counts, pdis_counts,hss_counts], axis=1).fillna(0).astype(int)

# Total unique students per year (across all modules using aadhar_no and barcode combined)
# Create a unified student_id for accurate counting
# Create unified student_id column for deduplication
df['student_id'] = df.apply(lambda row: row['barcode'] if row['module'] == 'PDIS' else row['aadhar_no'], axis=1)

# Count unique students per year
unique_students = df.groupby('academic_year')['student_id'].nunique().rename('Total Students')

# Module Enrollments (sum of all modules, including duplicates)
module_enrollments = combined.sum(axis=1).rename('Module Enrollments')


# Final year-wise summary table
summary = pd.concat([combined, module_enrollments, unique_students], axis=1).reset_index()

# Add totals row
totals = summary.drop(columns='academic_year').sum(numeric_only=True)
totals['academic_year'] = 'Total'

# Append total to the summary
summary_with_total = pd.concat([summary, pd.DataFrame([totals])], ignore_index=True)

# Display the result
print("========== [YEAR-WISE STUDENT COUNTS] ==========")
print(summary_with_total.to_string(index=False))

print()



print("========== [STUDENTS IN MULTIPLE YEARS] ==========")

# For ITI
iti_years_per_student = df_iti.groupby('aadhar_no')['academic_year'].nunique()

# Calculate total extra contributions from students who appeared in multiple years
extra_counts = (df_iti.groupby('aadhar_no')['academic_year'].nunique() - 1).clip(lower=0).sum()
print(f"Extra ITI student counts added across years (due to repeats): {extra_counts}")

# For Diploma
diploma_years_per_student = df_diploma.groupby('aadhar_no')['academic_year'].nunique()
diploma_extra_counts = (diploma_years_per_student - 1).clip(lower=0).sum()
print(f"Extra Diploma student counts added across years (due to repeats): {diploma_extra_counts}")

# For PDIS (barcode is used as unique identifier here)
pdis_years_per_student = df_pdis.groupby('barcode')['academic_year'].nunique()
pdis_extra_counts = (pdis_years_per_student - 1).clip(lower=0).sum()
print(f"Extra PDIS student counts added across years (due to repeats): {pdis_extra_counts}")


# For HSS (barcode is used as unique identifier here)
hss_years_per_student = hss_df.groupby('aadhar_no')['academic_year'].nunique()
hss_extra_counts = (hss_years_per_student - 1).clip(lower=0).sum()
print(f"Extra HSS student counts added across years (due to repeats): {hss_extra_counts}")

========== [STUDENT OVERVIEW] ==========
Total ITI Students     : 529,085
Total Diploma Students : 417,756
Total PDIS Students    : 5,717
Total HSS Students    : 2,643,387

========== [YEAR-WISE STUDENT COUNTS] ==========
academic_year    ITI  Diploma  PDIS     HSS  Module Enrollments  Total Students
         2017  27337        0     0       0               27337           27337
         2018  70613    52530     0  367581              490724          119675
         2019  64148    44080     0  355308              463536          105304
         2020  67412    56533   916  341822              466683          121737
         2021  81948    59278   874  375368              517468          137672
         2022  74104    78196  1187  417040              570527          148698
         2023  92085    79700  1287  419135              592207          167610
         2024  83958    81789  1453  473097              640297          161490
        Total 561605   452106  5717 2749351             37

In [13]:
# Step 1: Count how many unique years each ITI student appeared in
iti_years_per_student = df_iti.groupby('aadhar_no')['academic_year'].nunique()

# Step 2: Count how many students fall into each "year span" (1 year, 2 years, etc.)
students_by_year_span = iti_years_per_student.value_counts().sort_index()

# Step 3: Calculate the number of extra appearances (repeats beyond first year)
extra_repeat_count = sum((years - 1) * num_students for years, num_students in students_by_year_span.items())

# Step 4: Display detailed breakdown
print(f"{'Year Span':<10} | {'Number of Students':<20} | {'Repeat Year Entries':<22}")
print("-" * 10 + " | " + "-" * 20 + " | " + "-" * 22)

for years, num_students in students_by_year_span.items():
    extra = (years - 1) * num_students
    print(f"{years:<10} | {num_students:<20} | {extra:<22}")

# Final summary
total_unique_students = len(iti_years_per_student)
yearwise_total_counts = df_iti.groupby('academic_year')['aadhar_no'].nunique().sum()
difference = yearwise_total_counts - total_unique_students

print("\nSummary:")
print(f"- Total Unique ITI Students        : {total_unique_students:,}")
print(f"- Total Students Across Years      : {yearwise_total_counts:,}")
print(f"- Total Extra Year Counts          : {extra_repeat_count:,}")
print(f"- Difference Matches Extra Count?  : {extra_repeat_count == difference}")

Year Span  | Number of Students   | Repeat Year Entries   
---------- | -------------------- | ----------------------
1          | 498967               | 0                     
2          | 27984                | 27984                 
3          | 1914                 | 3828                  
4          | 182                  | 546                   
5          | 30                   | 120                   
6          | 6                    | 30                    
7          | 2                    | 12                    

Summary:
- Total Unique ITI Students        : 529,085
- Total Students Across Years      : 561,605
- Total Extra Year Counts          : 32,520
- Difference Matches Extra Count?  : True


In [11]:
conn = sqlite3.connect(db_path)

query = """
SELECT
    academic_year,
    aadhar_no,
    barcode,
    module,
    option_data
FROM students
"""

json_df = pd.read_sql_query(query, conn)
conn.close()


# Function to parse JSON strings safely
def parse_option_data(val):
    """
    Safely parses a JSON string into a Python object.

    Parameters:
    ----------
    val : str, list, or any
        The value to be parsed. If it's a JSON string, it will be decoded.
        If it's already a list, it is returned unchanged.
        All other types return an empty list.

    Returns:
    -------
    list
        Parsed list from JSON string, or the original list, or an empty list if parsing fails.
    """
    if isinstance(val, str):
        try:
            return json.loads(val)
        except json.JSONDecodeError:
            return []
    elif isinstance(val, list):
        return val
    else:
        return []

json_df['option_data'] = json_df['option_data'].apply(parse_option_data)
df_exploded = json_df.explode('option_data').reset_index(drop=True)
option_details = pd.json_normalize(df_exploded['option_data'])
json_df = pd.concat([df_exploded.drop(columns=['option_data']), option_details], axis=1)

json_df.head()

,academic_year,aadhar_no,barcode,module,Phase,Option_No,Institute_Name,Trade,Status
0,2020,NA,20S00045,PDIS,NaN,NaN,NaN,NaN,NaN
1,2020,NA,20S00031,PDIS,NaN,NaN,NaN,NaN,NaN
2,2020,NA,20S000532,PDIS,1,1,SKDAV Government Polytechnic Rourkela,Post Diploma in Industrial Safety,Opted
3,2020,NA,20S000532,PDIS,1,2,"Asian Workers Development institute (AWDI),Rou...",Post Diploma in Industrial Safety,Opted
4,2020,NA,20S000532,PDIS,1,3,"Koel Institute of Industrial Safety,Rourkela",Post Diploma in Industrial Safety,Opted


In [ ]:
json_df.columns

Index(['academic_year', 'aadhar_no', 'barcode', 'module', 'Phase', 'Option_No',
       'Institute_Name', 'Trade', 'Status'],
      dtype='object')

In [35]:
option_df = pd.read_parquet(RAW_DATA_DIR / "hss_option.pq", engine='pyarrow')

In [38]:
option_df.columns

Index(['barcode', 'ReportedInstitute', 'SAMSCode', 'Stream',
       'InstituteDistrict', 'InstituteBlock', 'TypeofInstitute', 'Phase',
       'Year', 'AdmissionStatus', 'OptionNo'],
      dtype='object')

In [36]:
option_df.columns

Index(['barcode', 'ReportedInstitute', 'SAMSCode', 'Stream',
       'InstituteDistrict', 'InstituteBlock', 'TypeofInstitute', 'Phase',
       'Year', 'AdmissionStatus', 'OptionNo'],
      dtype='object')

In [19]:
option_df.columns

Index(['barcode', 'ReportedInstitute', 'SAMSCode', 'Stream',
       'InstituteDistrict', 'InstituteBlock', 'TypeofInstitute', 'Phase',
       'Year', 'AdmissionStatus', 'OptionNo'],
      dtype='object')

In [31]:
# Step 1: Select only the necessary columns from hss_df
hss_subset = hss_df[['barcode', 'aadhar_no', 'academic_year']]

# Step 2: Merge option_df with hss_subset on 'barcode'
merged_df = option_df.merge(hss_subset, on='barcode', how='left')

# Step 3: Rename columns as required
merged_df = merged_df.rename(columns={
    'ReportedInstitute': 'reported_institute',
    'SAMSCode': 'sams_code',
    'Stream': 'stream',
    'TypeofInstitute': 'type_of_institute',
    'Phase': 'phase',
    'Year': 'year',
    'AdmissionStatus': 'admission_status',
    'OptionNo': 'option_no',
    'barcode': 'barcode'
})

# Step 4: Select only the desired final columns
hss_first_choice_admissions = merged_df[
    ['reported_institute', 'sams_code', 'stream', 'type_of_institute',
     'phase', 'admission_status', 'option_no',
     'barcode', 'academic_year', 'aadhar_no']
]


In [32]:
hss_first_choice_admissions.columns

Index(['reported_institute', 'sams_code', 'stream', 'type_of_institute',
       'phase', 'admission_status', 'option_no', 'barcode', 'academic_year',
       'aadhar_no'],
      dtype='object')

In [33]:
# save df in parquet format
from pathlib import Path
DATA_DIR = Path("C:/Users/Admin/Documents/GitHub/sams/data")
RAW_DATA_DIR = DATA_DIR / "raw"
output_path = Path(RAW_DATA_DIR) / "hss_first_choice_admissions.pq"
hss_first_choice_admissions.to_parquet(output_path, index=False, compression='snappy')

In [34]:
hss_first_choice_admissions.head()

,reported_institute,sams_code,stream,type_of_institute,phase,admission_status,option_no,barcode,academic_year,aadhar_no
0,"Gangadhar Meher Higher Secondary School, Samba...",28141101,Commerce,Govt. (SME),1,TC TAKEN,1,18J0000045,2018,<NA>
1,Netaji Subash Chandra Bose Higher Secondary Sc...,28111101,Commerce,Govt. (SME),1,NOT SELECTED,2,18J0000045,2018,<NA>
2,"Hirakud Higher Secondary School, Hirakud",28132101,Commerce,Non-Govt Aided (488 Categories),1,NOT SELECTED,3,18J0000045,2018,<NA>
3,"Surajmal Higher Secondary School, Rampella",28112102,Commerce,Non-Govt Aided (488 Categories),1,NOT SELECTED,4,18J0000045,2018,<NA>
4,"Burla N.A.C. Higher Secondary School, Burla",28122101,Commerce,Non-Govt Aided (488 Categories),1,NOT SELECTED,5,18J0000045,2018,<NA>


In [13]:
option_df = option_df.rename(columns={'Year': 'academic_year', 'OptionNo': 'Option_No'})

In [14]:
# Merge on barcode
hss_merged = pd.merge(
    hss_df[['aadhar_no', 'barcode', 'academic_year']],
    option_df[['barcode', 'Option_No']],
    on='barcode',
    how='inner'  # use 'inner' to keep only matching barcodes
)

# Display result
hss_merged.columns

Index(['aadhar_no', 'barcode', 'academic_year', 'Option_No'], dtype='object')

In [15]:
import pandas as pd

# ========== [APPLICATIONS PER STUDENT SUMMARY (BASED ON Option No)] ==========


import pandas as pd

# ========== [APPLICATIONS PER STUDENT SUMMARY (BASED ON Option No)] ==========

def summarize_applications(df, option_col='Option_No', module=None, exclude_zero=False):
    data = df.copy()

    # Create unified student identifier
    data['student_id'] = data.apply(
        lambda row: row['barcode'] if row['module'] == 'PDIS' else row['aadhar_no'], axis=1
    )
    data = data.dropna(subset=['student_id'])

    # Count options per student per year
    app_counts = data.groupby(['academic_year', 'student_id'])[option_col].count()

    # Optionally exclude students with zero applications
    if exclude_zero:
        app_counts = app_counts[app_counts > 0]

    # Aggregate statistics by academic year
    summary = app_counts.groupby(level=0).agg([
        ('num_students', 'count'),
        ('avg_apps', 'mean'),
        ('std_dev', 'std'),
        ('p25', lambda x: x.quantile(0.25)),
        ('median', 'median'),
        ('p75', lambda x: x.quantile(0.75))
    ]).round(2).reset_index()

    title = f"[APPLICATION STATS - {module.upper()}]" if module else "[APPLICATION STATS - ALL MODULES + PDIS]"
    if exclude_zero:
        title += " (Excl. 0-applicants)"

    print(f"\n========== {title} ==========")
    print(summary.to_string(index=False))
    return summary


# 2.1 All Students Combined (Includes ITI, Diploma, HSS, PDIS)
# All Students Combined (Includes ITI, Diploma, HSS, PDIS)
summarize_applications(json_df)

# ITI Students Only (excluding zero-applicants)
summarize_applications(json_df[json_df['module'] == 'ITI'], module='ITI', exclude_zero=True)

# Diploma Students Only
summarize_applications(json_df[json_df['module'] == 'Diploma'], module='Diploma')

# HSS Students Only
hss_merged['module'] = 'HSS'
summarize_applications(hss_merged, module='HSS')


========== [APPLICATION STATS - ALL MODULES + PDIS] ==========
 academic_year  num_students  avg_apps  std_dev  p25  median  p75
          2017         27337      5.24    76.48  2.0     4.0  6.0
          2018        119675      3.00    48.71  0.0     1.0  3.0
          2019        105304      3.85    65.61  0.0     1.0  5.0
          2020        121737      2.78     4.68  0.0     1.0  5.0
          2021        137672      3.09     4.68  0.0     1.0  5.0
          2022        148698      3.35     5.15  0.0     1.0  5.0
          2023        167610      3.67     4.96  0.0     2.0  5.0
          2024        161490      4.03     5.57  1.0     2.0  5.0

========== [APPLICATION STATS - ITI] (Excl. 0-applicants) ==========
 academic_year  num_students  avg_apps  std_dev  p25  median  p75
          2017         26362      5.43    77.88  2.0     4.0  6.0
          2018         29115      5.85     5.68  1.0     4.0  8.0
          2019         32977      6.98     3.93  5.0     5.0  8.0
        

,academic_year,num_students,avg_apps,std_dev,p25,median,p75
0,2018,365014,6.06,408.81,5.0,5.0,5.0
1,2019,352732,5.71,250.89,5.0,5.0,5.0
2,2020,339415,6.56,471.97,5.0,5.0,5.0
3,2021,372655,6.88,727.05,5.0,5.0,5.0
4,2022,414021,7.12,637.90,5.0,5.0,6.0
5,2023,416077,7.41,661.38,5.0,5.0,6.0
6,2024,469592,6.41,115.12,5.0,5.0,6.0
